# Programming Assignment: Choosing the number of neighbors 

This week we will start working with metric classifiers. In this assignment we will focus on closest **k-neighbours method** and attune parameter k
In the next paragraph, original six tasks will be presented in Russian. 

**Tasks will be duplicated in english further in this assignment**
    
1. Загрузите выборку Wine по адресу https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data (файл также приложен к этому заданию)
2. Извлеките из данных признаки и классы. Класс записан в первом столбце (три варианта), признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine (см. также файл wine.names, приложенный к заданию)
3. Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).
4. Найдите точность классификации на кросс-валидации для метода k ближайших соседей (sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50. При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)? Данные результаты и будут ответами на вопросы 1 и 2.
5. Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale. Снова найдите оптимальное k на кросс-валидации.
6. Какое значение k получилось оптимальным после приведения признаков к одному масштабу? Приведите ответы на вопросы 3 и 4. Помогло ли масштабирование признаков?

In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale
import pandas

# Task №1

*Ru*:Загрузите выборку Wine по адресу https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data 

*Eng*: Download the Wine sample

In [2]:
df = pandas.read_csv('wine.data', header=None)

In [3]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


# Task №2

*Ru*:Извлеките из данных признаки и классы. Класс записан в первом столбце (три варианта), признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine (см. также файл wine.names, приложенный к заданию)

*Eng*: Extract features and classes from the data. The class is written in the first column (three options), features - in columns from the second to the last. More details about data may be found there https://archive.ics.uci.edu/ml/datasets/Wine 

In [4]:
Y = df[0].copy(deep=False) 

In [5]:
Y

0      1
1      1
2      1
3      1
4      1
      ..
173    3
174    3
175    3
176    3
177    3
Name: 0, Length: 178, dtype: int64

In [6]:
#X = df['14.23':].copy(deep=False)
X = df.loc[:, 1:]

In [7]:
X

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


## Task №3
*Ru*: Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).

*Eng*: Quality assessment should be carried out by cross-validation in 5 blocks (5-fold). Create a splitter generator that shuffles the sample before forming the blocks (shuffle = True). For reproducible results, create a KFold generator with a fixed parameter random_state = 42. Use accuracy as a measure of quality. 

In [8]:
cv = KFold(n_splits = 5,shuffle=True,  random_state=42)

## Task №4
*Ru*: Найдите точность классификации на кросс-валидации для метода k ближайших соседей (sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50. При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)? Данные результаты и будут ответами на вопросы 1 и 2.

*Eng*: Find the classification accuracy for cross-validation for the k nearest neighbors method (sklearn.neighbors.KNeighborsClassifier), for k from 1 to 50. At what k is the optimal quality? What is it equal to (a number between 0 and 1)? These results will be the answers to questions 1 and 2. 

let's try to do that will k = 5

In [9]:
k = 5

In [10]:
KNN = KNeighborsClassifier(k)

In [11]:
cross_val_score(KNN, X, Y, cv = cv,  scoring = 'accuracy')

array([0.72222222, 0.61111111, 0.61111111, 0.68571429, 0.74285714])

In [12]:
result = []
for k in range(1, 51):
    KNN = KNeighborsClassifier(k)
    result.append(cross_val_score(KNN, X, Y, cv = cv,  scoring = 'accuracy'))

In [13]:
result

[array([0.77777778, 0.66666667, 0.72222222, 0.71428571, 0.77142857]),
 array([0.72222222, 0.63888889, 0.69444444, 0.57142857, 0.68571429]),
 array([0.80555556, 0.61111111, 0.63888889, 0.65714286, 0.82857143]),
 array([0.75      , 0.52777778, 0.61111111, 0.65714286, 0.74285714]),
 array([0.72222222, 0.61111111, 0.61111111, 0.68571429, 0.74285714]),
 array([0.72222222, 0.63888889, 0.63888889, 0.68571429, 0.68571429]),
 array([0.69444444, 0.58333333, 0.72222222, 0.68571429, 0.71428571]),
 array([0.72222222, 0.61111111, 0.66666667, 0.65714286, 0.74285714]),
 array([0.72222222, 0.63888889, 0.72222222, 0.68571429, 0.74285714]),
 array([0.72222222, 0.61111111, 0.63888889, 0.68571429, 0.74285714]),
 array([0.75      , 0.63888889, 0.66666667, 0.71428571, 0.74285714]),
 array([0.72222222, 0.66666667, 0.69444444, 0.68571429, 0.71428571]),
 array([0.72222222, 0.69444444, 0.66666667, 0.65714286, 0.71428571]),
 array([0.72222222, 0.66666667, 0.72222222, 0.6       , 0.68571429]),
 array([0.75      , 

In [14]:
pandas.DataFrame(result).mean(axis =1 ).sort_values(ascending = False).head(1)

0    0.730476
dtype: float64

## Task №5
*Ru*: Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale. Снова найдите оптимальное k на кросс-валидации.

*Eng*: Scale the features using the sklearn.preprocessing.scale function. Find the optimal k again on cross-validation 

In [15]:
X = scale(X)

In [16]:
scores = list()
for k in range(1,51):
    KNN = KNeighborsClassifier(k)
    scores.append(cross_val_score(KNN, X, Y, cv = cv, scoring = 'accuracy'))
pandas.DataFrame(scores, range(1, 51)).mean(axis =1 ).sort_values(ascending = False).head(1)

29    0.977619
dtype: float64

## Task №6
*Ru*: Какое значение k получилось оптимальным после приведения признаков к одному масштабу? Приведите ответы на вопросы 3 и 4. Помогло ли масштабирование признаков?

*Eng*: What value of k turned out to be optimal after reducing the features to the same scale? Please provide answers to questions 3 and 4. Did feature scaling help? 

before scaling: k = 1, accuracy = 0.730476  
after scaling: k = 29, accuracy = 0.977619